In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Read pickle save
with open("../tracker_saves/ball_detections.pkl", "rb") as f:
    ball_positions = pickle.load(f)

In [ ]:
ball_positions = [x.get(1, []) for x in ball_positions]
df_ball = pd.DataFrame(ball_positions, columns=['x1', 'y1', 'x2', 'y2'])

df_ball = df_ball.interpolate()
df_ball = df_ball.bfill()

In [ ]:
df_ball["mid_y"] = (df_ball["y1"] + df_ball["y2"]) / 2
df_ball["mid_y_rolling_mean"] = df_ball["mid_y"].rolling(window=5, min_periods= 1, center=False).mean()

In [ ]:
plt.plot(df_ball["mid_y_rolling_mean"], label="Mid Y Position")

In [ ]:
df_ball["delta_y"] = df_ball["mid_y_rolling_mean"].diff().fillna(0)

In [ ]:
plt.plot(df_ball["delta_y"], label="Mid Y Position")

In [ ]:
df_ball["ball_hit"] = 0

In [ ]:
minimum_change_frames_for_hit = 30
for i in range(1,len(df_ball)- int(minimum_change_frames_for_hit*1.2) ):
    negative_position_change = df_ball['delta_y'].iloc[i] > 0 and df_ball['delta_y'].iloc[i+1] < 0
    positive_position_change = df_ball['delta_y'].iloc[i] < 0 and df_ball['delta_y'].iloc[i+1] > 0

    if negative_position_change or positive_position_change:
        change_count = 0 
        for change_frame in range(i+1, i+int(minimum_change_frames_for_hit*1.2)+1):
            negative_position_change_following_frame = df_ball['delta_y'].iloc[i] > 0 and df_ball['delta_y'].iloc[change_frame] < 0
            positive_position_change_following_frame = df_ball['delta_y'].iloc[i] < 0 and df_ball['delta_y'].iloc[change_frame] > 0

            if negative_position_change and negative_position_change_following_frame:
                change_count+=1
            elif positive_position_change and positive_position_change_following_frame:
                change_count+=1
    
        if change_count>minimum_change_frames_for_hit-1:
            df_ball['ball_hit'].iloc[i] = 1

frame_nums_with_ball_hits = df_ball[df_ball['ball_hit']==1].index.tolist()
    

In [ ]:
df_ball[df_ball['ball_hit']==1]